In [15]:
import string
import numpy as np

In [16]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    embedding = np.zeros(n_classes)
    embedding[idx] = 1.0
    return embedding


def word2seq(word: str, alphabet: list) -> np.ndarray:
    return np.array([one_hot(len(alphabet), alphabet.index(c)) for c in word])


def seq2word(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(x)] for x in seq])

In [17]:
alphabet = string.ascii_letters + " " + string.punctuation + string.digits
n_symbols = len(alphabet)
print(n_symbols, alphabet)

95 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [18]:
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

In [19]:
class EncoderRNN:
    def __init__(self, n_in: int, n_h: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)
    
    def forward(self, sequence: np.ndarray) -> np.ndarray:
        context = np.zeros(len(self.V))
        for i,x in enumerate(sequence):
            context = np.tanh(self.U @ x + self.V @ context + self.d)
        return context

In [20]:
class DecoderRNN:
    def __init__(self, n_h: int, n_out: int) -> None:
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)

        self.W: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.b: np.ndarray = np.zeros(n_out)
    
    def forward(self, context: np.ndarray, t: int) -> np.ndarray:
        outputs = np.zeros((t, self.b.size))
        for i in range(t):
            context = np.tanh(self.V @ context + self.d)
            outputs[i] = softmax(self.W @ context + self.b)
        return outputs

In [21]:
n_h = 32

encoder = EncoderRNN(n_symbols, n_h)
decoder = DecoderRNN(n_h, n_symbols)

in_text = 'the quick brown fox jumps over the lazy dog'

x = word2seq(in_text, alphabet)
t = 128

context = encoder.forward(x)
output = decoder.forward(context, t)

out_text = seq2word(output, alphabet)
print(out_text)

]'I2gV+%n+g1rNr+y\ro.%$9Ho3xdr{\k)'fV{0)rSxr`{GE@GO\~yirR3VGonF>-rNc>yNrg9aqM|;A'WM/(IA"yI'"8@i"-`"@@24qivRR6O]'L@= =pI@)Yp6@=?)
